In [ ]:
from os import path

# Third-party
from astropy.table import Table
import astropy.coordinates as coord
import astropy.units as u
from astropy.constants import G, c
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.stats import scoreatpercentile

from comoving_rv.log import logger
from comoving_rv.db import Session, Base, db_connect
from comoving_rv.db.model import (Run, Observation, TGASSource, SimbadInfo, PriorRV,
                                  SpectralLineInfo, SpectralLineMeasurement, RVMeasurement)

import matplotlib as mpl
mpl.rc('text', usetex=True)
mpl.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]

In [ ]:
base_path = '/Volumes/ProjectData/gaia-comoving-followup/'
db_path = path.join(base_path, 'db.sqlite')
engine = db_connect(db_path)
session = Session()

In [ ]:
tbl = Table.read('group_llr_dv_tbl.ecsv', format='ascii.ecsv')

In [ ]:
dllr = tbl['R_RV'] - tbl['R_mu']
print('{0} are comoving'.format((dllr>0).sum()))

In [ ]:
np.random.seed(1234)

fig,ax = plt.subplots(1, 1, figsize=(5, 5))

ax.scatter(dllr, tbl['dv_50'], marker='.', alpha=0.7, linewidth=0)

ax.set_xlim(-30, 10)
ax.set_xlabel(r'$\mathcal{R}_{\rm RV} - \mathcal{R}_\mu$')
ax.set_ylabel(r'$|\boldsymbol{v}_1 - \boldsymbol{v}_2|$ ' + 
              '[{0}]'.format((u.km/u.s).to_string('latex_inline')))
ax.set_ylim(-2, 65)

# for a few random pairs, show error bars
bin_edges = np.arange(-30, 10+1, 5)
idxs = []
for l,r in zip(bin_edges[:-1], bin_edges[1:]):
    idx, = np.where((dllr > l) & (dllr < r))
    idx = np.random.choice(idx, size=1, replace=False)
    idxs.append(idx)
idxs = np.ravel(idxs)
    
ax.errorbar(dllr[idxs], tbl['dv_50'][idxs], 
            yerr=[tbl['dv_15'][idxs], tbl['dv_85'][idxs]], 
            marker='None', ecolor='#aaaaaa', linestyle='none', zorder=-10,
            elinewidth=1.)

fig.tight_layout()
fig.savefig('llr-vs-dv.pdf')

In [ ]:
all_color = '#888888'
comoving_color = '#000000'
alpha = 0.75

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,4))

_,bins,_ = axes[0].hist(tbl['sep_3d'], bins='auto', 
                        color=all_color, alpha=alpha, label='all observed')
axes[0].hist(tbl['sep_3d'][tbl['R_RV'] > tbl['R_mu']], bins=bins, 
             color=comoving_color, alpha=alpha, 
             label=r'$\mathcal{R}_{\rm RV} > \mathcal{R}_\mu$')
axes[0].set_xlabel('separation [pc]')
axes[0].set_xlim(0, 10)
axes[0].xaxis.set_ticks(np.arange(0,10+1,2))
axes[0].set_ylabel('$N$')
axes[0].legend(loc='upper left', fontsize=18)

_,bins,_ = axes[1].hist(tbl['sep_2d'], bins=np.logspace(-3, 1, 11), color=all_color, alpha=alpha)
axes[1].hist(tbl['sep_2d'][tbl['R_RV'] > tbl['R_mu']], bins=bins, color=comoving_color, alpha=alpha)

axes[1].set_xlabel('sky separation [deg]')
axes[1].set_xlim(1e-3, 1e1)

axes[1].axvline((10*u.arcsecond).to(u.degree).value, linestyle='--',
                color='#aaaaaa', zorder=-10)
axes[1].text(0.0024, 100, '10 arcsec', ha='center', va='center', 
             rotation=90, fontsize=14, color='#555555')

axes[1].set_xscale('log')
axes[1].set_yscale('log')

fig.tight_layout()
fig.savefig('separation-hist.pdf')

In [ ]:
((tbl['sep_3d'] > 1) & (dllr > 0)).sum() / (tbl['sep_3d'] > 1).sum()

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(6.5,5))

plt.scatter(tbl['sep_3d'], tbl['dv'], marker='.', linewidth=0, alpha=alpha,
            color=all_color, s=15, label='all observed')
plt.scatter(tbl['sep_3d'][tbl['R_RV'] > tbl['R_mu']], tbl['dv'][tbl['R_RV'] > tbl['R_mu']], 
            marker='o', linewidth=0, alpha=alpha, color=comoving_color, s=10,
            label=r'$\mathcal{R}_{\rm RV} > \mathcal{R}_\mu$')
plt.xscale('log')
plt.xlim(1E-2, 2E1)
plt.ylim(-5, 100)

ax.set_xlabel(r'$|\boldsymbol{x}_1 - \boldsymbol{x}_2|$ ' + 
              '[{0}]'.format((u.pc).to_string('latex_inline')))
ax.set_ylabel(r'$|\boldsymbol{v}_1 - \boldsymbol{v}_2|$ ' + 
              '[{0}]'.format((u.km/u.s).to_string('latex_inline')))

ax.legend(loc='upper left', fontsize=18)

fig.tight_layout()
fig.savefig('dx-dv.pdf')

### Debugging

In [ ]:
plt.hist(tbl['R_mu'])

In [ ]:
np.random.seed(1234)

fig,ax = plt.subplots(1, 1, figsize=(5.5, 5))
cs = ax.scatter(tbl['R_mu'], dllr, marker='o', c=tbl['sep_3d'],
                alpha=0.7, linewidth=0, s=15, cmap='plasma_r')
ax.set_ylim(-30, 10)
ax.set_xlabel('old likelihood ratio')
ax.set_ylabel('(new - old)')
cb = fig.colorbar(cs)
cb.set_label('sep [pc]')

fig,ax = plt.subplots(1, 1, figsize=(5.5, 5))
cs = ax.scatter(tbl['R_mu'], dllr, marker='o', c=tbl['sep_2d'],
                alpha=0.7, linewidth=0, s=15, cmap='plasma_r')
ax.set_ylim(-30, 10)
ax.set_xlabel('old likelihood ratio')
ax.set_ylabel('(new - old)')
cb = fig.colorbar(cs)
cb.set_label('sep [deg]')